# 基本思路：
    1、获取stockbasic和stockearningreport的全量数据
    2、stockbasic根据所在市场进行过滤，只要主板和中小板的
    3、stockearingreport进行筛选，条件为：
        3.1 基于2中的主板和中小板数据
        3.2 只要最近3年的数据
        3.3 仅要年报（1231）这一天
    4、完成后，stockearningreport根据code进行聚类，并获取net_proft_yoy_raise的最小，均值和最大值
    5、根据group后的net_proft_yoy_raise最小值进行筛选，暂定条件为大于30%

In [4]:
import os
import sys

mod_path = os.path.abspath('./../dataUpdate/')
print(mod_path)
sys.path.append('/Users/roland/vscode_program/backtrader_learning')
sys.path.insert(0,'/Users/roland/vscode_program/backtrader_learning/dataUpdate')

/Users/roland/vscode_program/backtrader_learning/dataUpdate


In [5]:
from sqlalchemy.orm import Session
from dataUpdate import engine
from dataUpdate.model.basicinfo import StockEarningReport,StockBasic
import pandas as pd

In [6]:
sess = Session(engine)

In [7]:
# 1、获取所有报表是数据
sql = sess.query(StockEarningReport)
res_df = pd.read_sql(sql.statement,sql.session.bind)
res_df.head(3)

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
0,1,000617,-0.04,3.046625e+08,10.749529,-47.6992,-10572685.25,-134.422353,-231.3498,-1.27,-0.100000,3.111316,2010,0331
1,2,000757,-0.08,1.319204e+08,101.189055,16.3653,-25948235.48,16.468424,-146.0735,0.00,-0.042300,12.714494,2010,0331
2,3,301041,NaN,7.030412e+07,NaN,NaN,4112270.94,NaN,NaN,NaN,-0.169109,15.906955,2010,0331


In [8]:
# 2、获取主板+中小板数据
sql2 = sess.query(StockBasic.code,StockBasic.symbol,StockBasic.name,StockBasic.circulating_market_cap,StockBasic.sw_l1).filter(StockBasic.market.in_(['主板','中小板']))
res_df2 = pd.read_sql(sql2.statement,sql2.session.bind)
res_df2.head(3)

,code,symbol,name,circulating_market_cap,sw_l1
0,sh.600000,600000,浦发银行,3137.740,银行I
1,sh.600004,600004,白云机场,253.492,交通运输I
2,sh.600006,600006,东风汽车,128.400,汽车I


In [9]:
# 3、仅获取主板和中小板的，3年年份的年报数据
annul_report = res_df[(res_df['year'].isin([2018,2019,2020])) & (res_df['date']=='1231') & (res_df['code'].isin(res_df2['symbol']))]
annul_report.shape

(9327, 14)

In [10]:
annul_report[annul_report['code']=='601919']

,id,code,earning_per_share,revenue,revenue_yoy_rise,revenue_sos_rise,net_profit,net_profit_yoy_rise,net_profit_sos_rise,return_on_equity,operating_cash_flow_per_share,gross_profit_ratio,year,date
192670,192671,601919,0.12,1.208295e+11,33.566485,4.4383,1.230026e+09,-53.792034,-55.2878,5.52,0.795865,8.462042,2018,1231
225298,225299,601919,0.56,1.510567e+11,25.016363,-0.1942,6.764105e+09,449.920000,428.7379,21.57,1.729460,10.742339,2019,1231
258685,258686,601919,0.81,1.712588e+11,13.373888,22.6643,9.927098e+09,46.760000,122.8038,25.05,3.673106,14.208792,2020,1231


In [11]:
# 4、根据code进行groupby，并计算年净利润增长率的最小，均值和最大值
annul_report_groupby = annul_report.groupby('code').agg({'net_profit_yoy_rise':['min','mean','max']})

In [12]:
# 5、strategy1: 最低净利润增长也要大于30
res1 = annul_report_groupby[annul_report_groupby['net_profit_yoy_rise']['min']>20]

In [13]:
res1.shape

(197, 3)

In [14]:
res1.head()

net_profit_yoy_rise                        
                       min        mean         max
code                                              
000009           40.860000   73.700000  119.750000
000020           25.090000  109.651962  238.155887
000037           21.061024   69.170341  157.120000
000049           25.120000   30.677186   33.551559
000090           20.220000   36.100641   58.170000

In [15]:
res2 = pd.merge(left=res1,right=res_df2,left_on='code',right_on='symbol',how='inner')

/Users/roland/anaconda3/envs/py374/lib/python3.7/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [16]:
res2.sort_values(by='circulating_market_cap',inplace=True,ascending=False)
res2.head()

,"(net_profit_yoy_rise, min)","(net_profit_yoy_rise, mean)","(net_profit_yoy_rise, max)",code,symbol,name,circulating_market_cap,sw_l1
144,22.285194,34.811731,49.580000,sh.601888,601888,中国国旅,7572.87,休闲服务I
8,29.380000,32.939113,36.267338,sz.000568,000568,泸州老窖,4597.78,食品饮料I
111,36.250000,103.936743,192.330230,sh.600031,600031,三一重工,4066.41,机械设备I
132,28.630000,46.341950,56.390000,sh.600809,600809,山西汾酒,3840.38,食品饮料I
67,53.280000,62.486099,73.130000,sz.002475,002475,立讯精密,3701.54,电子I


In [17]:
res2['sw_l1'].value_counts()

机械设备I    25
化工I      24
医药生物I    18
电子I      16
公用事业I    13
计算机I     12
食品饮料I    11
电气设备I    10
房地产I      9
轻工制造I     9
建筑装饰I     6
建筑材料I     6
有色金属I     5
传媒I       5
家用电器I     4
交通运输I     4
采掘I       3
汽车I       3
通信I       2
纺织服装I     2
非银金融I     2
综合I       2
商业贸易I     2
国防军工I     2
休闲服务I     1
钢铁I       1
Name: sw_l1, dtype: int64

In [19]:
res2[res2.sw_l1=='机械设备I']

,"(net_profit_yoy_rise, min)","(net_profit_yoy_rise, mean)","(net_profit_yoy_rise, max)",code,symbol,name,circulating_market_cap,sw_l1
111,36.250000,103.936743,192.330230,sh.600031,600031,三一重工,4066.4100,机械设备I
139,54.930000,82.620255,119.050765,sh.601100,601100,恒立液压,1660.9400,机械设备I
5,51.649601,78.206534,116.420000,sz.000157,000157,中联重科,973.2520,机械设备I
168,50.820000,54.659607,61.198822,sh.603638,603638,艾迪精密,475.7010,机械设备I
65,24.830000,35.455504,50.850000,sz.002444,002444,巨星科技,411.7610,机械设备I
52,24.230000,317.654850,807.574551,sz.002353,002353,杰瑞股份,318.4180,机械设备I
157,21.551478,51.500493,78.670000,sh.603308,603308,应流股份,160.6860,机械设备I
6,28.740000,62.436612,127.699836,sz.000528,000528,柳工,132.9530,机械设备I
85,32.000000,47.073713,68.670000,sz.002698,002698,博实股份,108.3730,机械设备I
186,29.200000,38.332990,53.490000,sh.603960,603960,克来机电,86.1823,机械设备I
